In [ ]:
!apt-get install openjdk-8-jdk-headless -qq

Selecting previously unselected package libxtst6:amd64.
(Reading database ... 122519 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u362-ga-0ubuntu1~20.04.1_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u362-ga-0ubuntu1~20.04.1) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.
Preparing to unpack .../openjdk-8-jdk-headless_8u362-ga-0ubuntu1~20.04.1_amd64.deb ...
Unpacking openjdk-8-jdk-headless:amd64 (8u362-ga-0ubuntu1~20.04.1) ...
Setting up libxtst6:amd64 (2:1.2.3-1) ...
Setting up openjdk-8-jre-headless:amd64 (8u362-ga-0ubuntu1~20.04.1) ...
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/orbd to provide /usr/bin/orbd (orbd) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/servertool to

In [ ]:
!pip install pyspark==3.1.2
!pip install findspark
!pip install graphframes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 22.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880756 sha256=864fd6e13afa82fa316931b71c8cd8e851a93df5df4e8951a444ea18e8ae7d92
  Stored in directory: /root/.cache/pip/wheels/ef/70/50/7882e1bcb5693225f7cc86698f10953201b48b3f36317c2d18
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/15

In [ ]:
import os
import urllib
import findspark
import time

import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.sql import SparkSession
from graphframes import GraphFrame
from pyspark.sql.functions import lit, col
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

In [ ]:
# Download GraphFrames JAR
jar_url = "https://repos.spark-packages.org/graphframes/graphframes/0.8.0-spark3.0-s_2.12/graphframes-0.8.0-spark3.0-s_2.12.jar"
jar_path = "/content/graphframes.jar"

if not os.path.exists(jar_path):
    urllib.request.urlretrieve(jar_url, jar_path)

In [ ]:
# Set the location of Spark installation
findspark.init("/usr/local/lib/python3.10/dist-packages/pyspark")

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("GraphFrames") \
    .config("spark.jars", jar_path) \
    .getOrCreate()

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/Final Project/'

Mounted at /content/drive


In [ ]:
distance_df = spark.read.csv(
    drive_path + '16C_Distance_Matrix.csv',
    header=True,
    inferSchema=True
)
distance_df.show()

+--------------+-------+--------+----------+---------+-------+-------+-------+-------+-------+--------------+-------+-------+---------+---------+----------+-------+----------+
|          city| Dallas|New York|Washington|Baltimore| Boston| Denver|Orlando|  Miami|Chicago|Salt Lake City|Seattle|Houston|Las Vegas|Nashville|Cincinnati|Detroit|Pittsburgh|
+--------------+-------+--------+----------+---------+-------+-------+-------+-------+-------+--------------+-------+-------+---------+---------+----------+-------+----------+
|        Dallas|    0.0| 1552.81|   1327.58|  1365.62|1758.93| 794.16|1087.72|1302.54| 951.43|       1242.67|2100.88| 238.08|  1229.11|   661.58|    934.18|1169.58|    1220.7|
|      New York|1552.26|     0.0|    229.65|   192.33| 213.14|1788.53|1087.11|1281.81| 798.92|        2186.8|2851.18|1634.86|  2537.23|   890.26|     645.1| 626.48|     374.9|
|    Washington|1328.53|  228.99|       0.0|    37.97| 434.68|1646.04| 857.95|1052.65| 697.97|       2085.85|2750.23|141

In [ ]:
distance_df = distance_df.select(distance_df.columns[:-3])
distance_df = distance_df.limit(distance_df.count() - 3)
distance_df.show()

+--------------+-------+--------+----------+---------+-------+-------+-------+-------+-------+--------------+-------+-------+---------+---------+
|          city| Dallas|New York|Washington|Baltimore| Boston| Denver|Orlando|  Miami|Chicago|Salt Lake City|Seattle|Houston|Las Vegas|Nashville|
+--------------+-------+--------+----------+---------+-------+-------+-------+-------+-------+--------------+-------+-------+---------+---------+
|        Dallas|    0.0| 1552.81|   1327.58|  1365.62|1758.93| 794.16|1087.72|1302.54| 951.43|       1242.67|2100.88| 238.08|  1229.11|   661.58|
|      New York|1552.26|     0.0|    229.65|   192.33| 213.14|1788.53|1087.11|1281.81| 798.92|        2186.8|2851.18|1634.86|  2537.23|   890.26|
|    Washington|1328.53|  228.99|       0.0|    37.97| 434.68|1646.04| 857.95|1052.65| 697.97|       2085.85|2750.23|1411.13|  2404.63|   666.53|
|     Baltimore|1365.61|   193.4|     38.34|      0.0| 399.09|1647.14| 895.81|1090.51| 699.07|       2086.94|2751.32|1448.21

In [ ]:
# Create vertices DataFrame
vertices = distance_df.select("city").withColumnRenamed("city", "id")

# Generate a string for the stack() function
num_cities = len(distance_df.columns) - 1
stack_expr = (
    "stack("
    + str(num_cities)
    + ", "
    + ", ".join([f"'{c}', `{c}`" for c in distance_df.columns if c != 'city'])
    + ") as (dst, distance)"
)

# Create edges DataFrame
edges = distance_df.selectExpr("city as src", stack_expr).filter("src != dst")

In [ ]:
g = GraphFrame(vertices, edges)

In [ ]:
# Set the configuration to allow untyped Scala UDFs
spark.conf.set("spark.sql.legacy.allowUntypedScalaUDF", "true")

In [ ]:
start_time = time.time()

# Collect vertices and edges into lists for efficient lookup
vertex_ids = [row[0] for row in g.vertices.select('id').collect()]
edge_data = g.edges.select('src', 'dst', 'distance').collect()

# Convert edges into a dictionary for quick lookup
edge_dict = {(edge['src'], edge['dst']): edge['distance'] for edge in edge_data}

min_distance = float('inf')


def find_all_paths(source, destination, visited, path, all_paths,
                   unvisited, current_distance=0):
    """
    Recursive function to find all paths between source and destination
    using depth-first search (DFS).

    Args:
        source (str): The source vertex.
        destination (str): The destination vertex.
        visited (set): Set of visited vertices.
        path (list): Current path from source to destination.
        all_paths (list): List to store all the found paths.
        unvisited (set): Set of unvisited vertices.
        current_distance (float): Current distance traveled.

    Returns:
        None (all_paths will be populated with the found paths)
    """
    visited.add(source)
    path.append(source)
    unvisited.remove(source)
    global min_distance

    for vertex in vertex_ids:
        if (source, vertex) in edge_dict:
            edge_distance = edge_dict[(source, vertex)]
            if current_distance + edge_distance > min_distance:
                continue
            if vertex not in visited:
                # Recursive call for unvisited vertex
                find_all_paths(vertex, destination, visited, path, all_paths,
                    unvisited.copy(), current_distance + edge_distance)
            elif vertex == destination and not unvisited:
                # Destination reached with all vertices visited
                path.append(destination)
                if current_distance + edge_distance < min_distance:
                    min_distance = current_distance + edge_distance
                all_paths.append((path[:],
                                  round(current_distance + edge_distance, 2)))
                path.pop()

    path.pop()
    visited.remove(source)
    unvisited.add(source)


end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", round(execution_time, 2), "seconds")

Execution time: 0.94 seconds


In [ ]:
start_time = time.time()

source = "Dallas"
destination = "Dallas"
cities = distance_df.columns[1:]

# Initialize visited, paths, and unvisited sets
visited = set()
paths = []
unvisited = set(cities)

# Find all paths from source to destination
find_all_paths(source, destination, visited, [], paths, unvisited)

# Convert paths into a PySpark DataFrame for better visualization
routes_data = [[path[0], path[1]] for path in paths]
routes_schema = StructType([
    StructField("Route", StringType(), nullable=False),
    StructField("Distance in miles", DoubleType(), nullable=False)
])
routes = spark.createDataFrame(routes_data, schema=routes_schema)

# Sort routes based on distance in ascending order
optimized_routes = routes.orderBy("Distance in miles")

end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", round(execution_time, 2), "seconds")

optimized_routes.show()

Execution time: 208.69 seconds
+--------------------+-----------------+
|               Route|Distance in miles|
+--------------------+-----------------+
|[Dallas, Las Vega...|          9131.01|
|[Dallas, Houston,...|          9132.49|
|[Dallas, Houston,...|          9156.94|
|[Dallas, Houston,...|          9157.81|
|[Dallas, Houston,...|          9206.48|
|[Dallas, Houston,...|          9210.65|
|[Dallas, Houston,...|          9213.47|
|[Dallas, Houston,...|          9214.72|
|[Dallas, Denver, ...|          9288.31|
|[Dallas, Denver, ...|          9304.62|
|[Dallas, Denver, ...|          9325.62|
|[Dallas, Denver, ...|          9340.52|
|[Dallas, Denver, ...|          9356.83|
|[Dallas, Denver, ...|          9377.83|
|[Dallas, Denver, ...|          9431.63|
|[Dallas, Denver, ...|          9444.63|
|[Dallas, Denver, ...|          9460.94|
|[Dallas, Denver, ...|          9481.94|
|[Dallas, Denver, ...|          9535.74|
|[Dallas, Denver, ...|          9556.74|
+--------------------+----

In [ ]:
optimized_routes.select(first('Route')).collect()[0][0]

'[Dallas, Las Vegas, Seattle, Salt Lake City, Denver, Chicago, Boston, New York, Baltimore, Washington, Miami, Orlando, Nashville, Houston, Dallas]'

In [ ]:
from pyspark.sql.functions import first
Route=optimized_routes.select(first('Route')).collect()[0][0].split("->")
df=pd.read_csv(drive_path + 'uscities.csv')
df = df[["city", "state_id", "lat","lng"]]
df

,city,state_id,lat,lng
0,New York,NY,40.6943,-73.9249
1,Los Angeles,CA,34.1141,-118.4068
2,Chicago,IL,41.8375,-87.6866
3,Miami,FL,25.7840,-80.2101
4,Dallas,TX,32.7935,-96.7667
...,...,...,...,...
30839,Ironville,PA,40.6586,-78.2155
30840,Newkirk,NM,35.0635,-104.2715
30841,Falcon Village,TX,26.5652,-99.1341
30842,Hidden Lakes,OH,40.5479,-82.7632


In [ ]:
Route=Route[0].strip('[]').split(', ')
Route

['Dallas',
 'Las Vegas',
 'Seattle',
 'Salt Lake City',
 'Denver',
 'Chicago',
 'Boston',
 'New York',
 'Baltimore',
 'Washington',
 'Miami',
 'Orlando',
 'Nashville',
 'Houston',
 'Dallas']

In [ ]:
cities = ['Dallas, TX', 'New York, NY', 'Washington, DC', 'Baltimore, MD', 'Boston, MA',
          'Denver, CO', 'Orlando, FL', 'Miami, FL', 'Chicago, IL',
          'Salt Lake City, UT', 'Seattle, WA', 'Houston, TX', 'Las Vegas, NV',
          'Nashville, TN', 'Cincinnati, OH', 'Detroit, MI', 'Pittsburgh, PA']

cities = [(c.split(',')[0].strip(), c.split(',')[1].strip()) for c in cities]
print(cities)

[('Dallas', 'TX'), ('New York', 'NY'), ('Washington', 'DC'), ('Baltimore', 'MD'), ('Boston', 'MA'), ('Denver', 'CO'), ('Orlando', 'FL'), ('Miami', 'FL'), ('Chicago', 'IL'), ('Salt Lake City', 'UT'), ('Seattle', 'WA'), ('Houston', 'TX'), ('Las Vegas', 'NV'), ('Nashville', 'TN'), ('Cincinnati', 'OH'), ('Detroit', 'MI'), ('Pittsburgh', 'PA')]


In [ ]:
cities_df = pd.DataFrame()  # Initialize an empty Pandas DataFrame
for city in cities:
    city_name = city[0]
    state_id = city[1]
    city_df = df[(df['city'] == city_name) & (df['state_id'] == state_id)]
    cities_df = pd.concat([cities_df, city_df])

In [ ]:
cities = {}
for index, row in cities_df.iterrows():
    if row['city'] in Route:
        cities[row['city']] = [row['lat'],row['lng']]

In [ ]:
import folium


# create a map centered on the US
us_map = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# add markers for the cities on the route
for i in Route:

  folium.Marker(location=cities[i], popup=i).add_to(us_map)
# for city, coord in cities.items():
#     folium.Marker(location=coord, popup=city).add_to(us_map)

# add a line connecting the cities on the route
locs = list(cities.values())
locs.append([32.7935, -96.7667])
folium.PolyLine(locations=locs, color="red", weight=5).add_to(us_map)

# display the map
us_map

In [ ]:
optimized_routes.show(10)

+--------------------+-----------------+
|               Route|Distance in miles|
+--------------------+-----------------+
|[Dallas, Las Vega...|          9131.01|
|[Dallas, Houston,...|          9132.49|
|[Dallas, Houston,...|          9156.94|
|[Dallas, Houston,...|          9157.81|
|[Dallas, Houston,...|          9206.48|
|[Dallas, Houston,...|          9210.65|
|[Dallas, Houston,...|          9213.47|
|[Dallas, Houston,...|          9214.72|
|[Dallas, Denver, ...|          9288.31|
|[Dallas, Denver, ...|          9304.62|
+--------------------+-----------------+
only showing top 10 rows

